### 支持向量机

- 数据集：python自带的乳腺癌数据集，因变量是0-1分类变量，共有569个样本。

In [1]:
# -*- coding: utf-8 -*-

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model,svm
from sklearn.model_selection import train_test_split

#### 第一步：得到训练集和测试集

In [ ]:
def get_data():
    '''
    加载出数据，数据是一个乳腺癌的数据
    :return: 训练样本集、测试样本集、训练样本集对应的标记、测试样本集对应的标记
    '''
    cancer=datasets.load_breast_cancer()
    X=cancer.data
    Y=cancer.target
    print("样本量:%s,特征量:%s"%(X.shape))
    return train_test_split(X, Y, test_size=0.25,random_state=0, stratify=Y)

X_train,X_test,y_train,y_test=get_data()

#### 第二步：线性支持向量机

In [3]:
def test_SVC_linear(*data):
    '''
    测试 SVC 的用法。这里使用的是最简单的线性核
    liear: u'*v
    :param data:  可变参数。它是一个元组，这里要求其元素依次为：训练样本集、测试样本集、训练样本的标记、测试样本的标记
    :return: None
    '''
    X_train,X_test,y_train,y_test=data
    cls=svm.SVC(kernel='linear')
    cls.fit(X_train,y_train)
    print('系数:%s, 截距：%s'%(cls.coef_,cls.intercept_))  #多分类采用1对1策略
    print('每个类别中支持向量机的个数：%s'%(cls.n_support_))
    print('支持向量机:%s'%cls.support_)
    print(cls.support_vectors_)
    print('测试精确度: %.2f' % cls.score(X_test, y_test))
    
test_SVC_linear(X_train,X_test,y_train,y_test) 

系数:[[ 0.95121247  0.05288539 -0.06865646  0.00225811 -0.2056804  -0.25361566
  -0.61281403 -0.32893896 -0.30207919 -0.0287923  -0.15970086  0.45401617
   1.07533084 -0.11680873 -0.04940714 -0.03038526 -0.16943876 -0.06710959
  -0.025793   -0.00653217 -0.23105979 -0.2327483  -0.1688175   0.00234639
  -0.43681359 -0.74946892 -1.46685738 -0.71658343 -0.66132248 -0.14260861]], 截距：[18.12104957]
每个类别中支持向量机的个数：[21 20]
支持向量机:[  5  18  29  42  90 114 120 146 147 181 197 218 229 246 289 297 306 337
 360 393 420  56  73  78  93 104 116 175 184 193 213 216 243 253 284 321
 344 356 382 387 388]
[[1.095e+01 2.135e+01 7.190e+01 ... 1.424e-01 2.964e-01 9.606e-02]
 [1.442e+01 1.977e+01 9.448e+01 ... 1.565e-01 2.718e-01 9.353e-02]
 [1.528e+01 2.241e+01 9.892e+01 ... 1.226e-01 3.175e-01 9.772e-02]
 ...
 [1.464e+01 1.685e+01 9.421e+01 ... 7.828e-02 2.455e-01 6.596e-02]
 [1.469e+01 1.398e+01 9.822e+01 ... 1.108e-01 2.827e-01 9.208e-02]
 [1.163e+01 2.929e+01 7.487e+01 ... 6.835e-02 2.884e-01 7.220e-02]]
测试精

#### 第三步：非线性支持向量机-多项式核函数
- 公式
    - poly:(gamma*u'*v+coef0)^degree

- 思路
    - 第一步：本来这一步想要选择gamma/coef0/degree来选择最佳的参数的，但是这个程序跑不出来(在电脑后台跑了1个多小时了)
    - 第二步：退而求其次将这三个参数设定好，一般使用模型默认的参数，degree=2，gamma='scale',coef0=0
- 结果
    - 运行时间很长，猜测是特征量为30个，2阶的多项式核函数导致待估参数太多了
    - 尝试跑一次，结果仍旧是卡住了，因此合理怀疑这个数据集不适合跑非线性支持向量机（作为对比，试着跑了一下鸢尾花的数据集，能够轻松跑出来）
    - 其他的核函数在经过尝试之后，也发现了同样的问题，因此这里不再过多的赘述。

In [ ]:
cls=svm.SVC(kernel='poly',degree=3)
cls.fit(X_train,y_train)
print('系数:%s, 截距：%s' % (cls.coef_, cls.intercept_))  # 多分类采用1对1策略 
print('每个类别中支持向量机的个数：%s' % (cls.n_support_))
print('支持向量机:%s' % cls.support_)
print(cls.support_vectors_)
print('测试误差: %.2f' % cls.score(X_test, y_test))

D:\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


#### 结论（一些个人思考）

- 选择用python做支持向量机的作业也是因为R语言我本身比较熟悉，R也不太适合跑数据量很大的SVM，于是想要学习一下python的SVM的实现
- 尝试过titanic数据集（含有因子类变量），直接用python的SVC函数训练会报错；用乳腺癌数据（全数值型变量，但特征个数较多有30个，但是训练样本量却只有426个左右），在使用非线性支持向量机的时候会因为参数过多导致无法优化得到参数估计（可能是存在梯度爆炸的问题）。
- 总而言之，对于数据量小的数据，可以选择简单的模型，防止过拟合和参数爆炸的问题